In [37]:
import nltk
#imports the list of stop words.
from nltk.corpus import stopwords
#Used to tokenize the text into a sentace or just words.
from nltk.tokenize import sent_tokenize, word_tokenize
#imort regular exprasions
import re
#Import text blob for sentiment demo
from textblob import TextBlob


import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  

stopWords = set(stopwords.words('english')) 


In [38]:
Sample_Sentiment_Data = pd.read_csv('SentiSample_10000.csv', encoding = "ISO-8859-1")

In [39]:
def textProcess(SentiText): 
    
    stopWords = set(stopwords.words('english')) 
    SentiText = SentiText.lower()                                     #convert to lowercase
    
    pattern = r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*' 
    SentiText= re.sub(pattern, "", SentiText)                         # remove url
    
    pattern = r"[\d]"                           
    SentiText= re.sub(pattern, "", SentiText)                         # remove numbers
    
    pattern = r"[^\w]"
    SentiText= re.sub(pattern, " ", SentiText)                        # remove commas and puntication
    
    SentiText = SentiText.strip()                                     # remove white spaces
    
    SentiText = word_tokenize(SentiText)                              # tokenize sentance
    
    SentiText = [word for word in SentiText if not word in stopWords] # remove Stop words
    return SentiText

In [40]:
length = 10000
i=0

count=0

for count in range (length):
    
    sentance = textProcess(Sample_Sentiment_Data.Message[count])
    palcer=""
    for i in (sentance):
        palcer = palcer + " " + i
    Sample_Sentiment_Data.loc[count,"Message"]= palcer

In [41]:
Sample_Sentiment_Data = Sample_Sentiment_Data.drop(columns=["Unnamed: 0"])

Sample_Sentiment_Data_processes = pd.read_csv('SentiSample_10000_processed.csv', encoding = "ISO-8859-1")
Sample_Sentiment_Data_processes = Sample_Sentiment_Data_processes.drop(columns=["Unnamed: 0"])

Sample_Sentiment_Data_processes=Sample_Sentiment_Data_processes.fillna("placeholder")

In [42]:
Sentiment_data_X = Sample_Sentiment_Data_processes[['Message']]

Sentiment_data_Y = Sample_Sentiment_Data_processes[['Sentiment']]

#split
from sklearn.model_selection import train_test_split
Sentiment_data_X, X_test, Sentiment_data_Y, y_test = train_test_split(Sample_Sentiment_Data_processes[['Message']], Sample_Sentiment_Data_processes[['Sentiment']], test_size = .30, shuffle = True)


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer=TfidfVectorizer()
vectorizer.fit(Sample_Sentiment_Data_processes.Message)
#results=vectorizer.fit_transform([document])
results=vectorizer.transform(Sentiment_data_X.Message)
resultsTest=vectorizer.transform(X_test.Message)
#results=vectorizer.fit_transform([document])
print(results)

  (0, 16639)	0.22917201315906666
  (0, 15186)	0.29129531619085847
  (0, 13778)	0.2730320398852412
  (0, 11669)	0.2871839077554137
  (0, 6521)	0.24396359128188672
  (0, 5221)	0.389624180226951
  (0, 4313)	0.263274889039997
  (0, 3760)	0.19238113314540017
  (0, 3605)	0.32499800298876563
  (0, 3017)	0.25414276448358253
  (0, 2256)	0.3362066854787182
  (0, 1066)	0.32358855070549614
  (1, 15974)	0.38731732158582943
  (1, 10766)	0.25059159554867605
  (1, 8661)	0.25700512389876645
  (1, 5996)	0.5073233968677964
  (1, 5594)	0.36477675571852836
  (1, 2273)	0.33364916296362734
  (1, 1147)	0.46837659683355676
  (2, 9170)	0.47165058858647535
  (2, 6199)	0.20920475943259828
  (2, 5234)	0.340866244924247
  (2, 4716)	0.4515568651774459
  (2, 206)	0.47165058858647535
  (2, 159)	0.43730014153344654
  :	:
  (6996, 6664)	0.33396173824454783
  (6996, 6429)	0.2573116577021671
  (6996, 5670)	0.31113901992012666
  (6996, 4064)	0.2887986461124082
  (6997, 15301)	0.3918422735216126
  (6997, 11153)	0.4380590316

In [44]:
import matplotlib.pyplot as plt  
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

In [45]:
Xtrain = results
X_test = resultsTest
ytrain = np.array(Sentiment_data_Y)
ytrain = np.hstack((ytrain))
print(ytrain)


[0 0 0 ... 0 4 0]


In [46]:


grid = SVC(kernel='rbf', C=32, gamma=0.0078125, degree=2)
grid.fit(Xtrain, ytrain)

#print("The best parameters are %s with a score of %0.2f"
#      % (grid.best_params_, grid.best_score_))

SVC(C=32, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma=0.0078125, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
y_prediction = grid.predict(X_test)

In [48]:
print(confusion_matrix(y_test, y_prediction))  
print(classification_report(y_test, y_prediction))

[[ 954  504]
 [ 342 1200]]
             precision    recall  f1-score   support

          0       0.74      0.65      0.69      1458
          4       0.70      0.78      0.74      1542

avg / total       0.72      0.72      0.72      3000



In [49]:
import pickle

In [50]:
filename = 'finalized_model_10000.sav'
pickle.dump(grid, open(filename, 'wb'))

In [51]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)


0.718


In [52]:
output = loaded_model.predict(X_test)
print(output)

[4 4 4 ... 4 4 4]
